In [ ]:
import pandas as pd
from os.path import isfile
from tqdm.auto import tqdm
from collections import defaultdict

In [ ]:
def compress_directory_to_zip(root_path, zip_name):
    import zipfile
    import os

    with zipfile.ZipFile(zip_name, 'w', compression=zipfile.ZIP_DEFLATED, compresslevel=9) as z:
        for root, dirs, files in os.walk(root_path):
            for file in files:
                z.write(os.path.join(root, file))
            for directory in dirs:
                z.write(os.path.join(root, directory))

In [ ]:
# df_resultados = pd.read_csv('data/resultados_mesas.csv')
df_resultados = pd.read_csv('data/resultados_mesas_full.csv')

# df_resultados = df_resultados.astype(dict.fromkeys(df_resultados.select_dtypes('int64').columns, 'int'))
# df_resultados.set_index('mesa_id', inplace=True)
print(len(df_resultados))

df_resultados = df_resultados[~(
    (df_resultados["massa"] == 0) &
    (df_resultados["bullrich"] == 0) &
    (df_resultados["milei"] == 0) &
    (df_resultados["bregman"] == 0) &
    (df_resultados["schiaretti"] == 0)
)]
print(len(df_resultados))

In [ ]:
# df_resultados.groupby(['distrito'])['distrito'].count()

In [ ]:
# df_mesas = pd.read_csv('data/mesas_por_localidad.csv', encoding='latin1')
# print(len(df_mesas.index))

# distrito_nombre = "Ciudad Autónoma de Buenos Aires"
# distrito_nombre = "Jujuy"
# distrito_nombre = "Río Negro"
# df_resultados = df_resultados[df_resultados['distrito'] == distrito_nombre]
# len(df_resultados)

df_resultados = df_resultados[df_resultados['distrito'].isin(["Ciudad Autónoma de Buenos Aires", "Jujuy", "Río Negro"])]
print(len(df_resultados.index))

In [ ]:
#left,right,top,bottom
cropping_values_mapping = {
    "Ciudad Autónoma de Buenos Aires": (.09, .4, .27, .48),
    "Jujuy": (.09, .57, .25, .5),
    "Río Negro": (.09, .59, .25, .47),
}

In [ ]:
from pathlib import Path
import shutil
import json
from PIL import Image as PilImage

shutil.rmtree('data/donut_dataset', ignore_errors=True)

types_dataset = ('train', 'validation', 'test')
types_dataset_paths = {}

for type_dataset in types_dataset:
    dataset_path = f'data/donut_dataset/{type_dataset}/'
    types_dataset_paths[type_dataset] = dataset_path
    Path(dataset_path).mkdir(parents=True, exist_ok=True)

In [ ]:
rows_to_sample_per_distrito = 300
distritos_disponibles = df_resultados['distrito'].unique().tolist()
df_resultados_sampled = []

for distrito in distritos_disponibles:
    df_resultados_sampled.append(df_resultados[df_resultados['distrito'] == distrito].sample(n=rows_to_sample_per_distrito, random_state=42))

df_resultados_sampled = pd.concat(df_resultados_sampled)
df_resultados_sampled = df_resultados_sampled.sample(frac=1, random_state=42).reset_index(drop=True)
df_resultados_sampled

In [ ]:
limit = df_resultados_sampled.shape[0]
# limit = 100 #1200
output_image_size = 640

telegrama_or_resultado_not_found = []

dataset_items = defaultdict(list)

for index, row in tqdm(df_resultados_sampled[:limit].iterrows(), total=limit):
    mesa_id = row['mesa_id']
    distrito = row['distrito']
    telegrama_image_path = f"data/telegramas/{distrito}/{mesa_id}.tiff"

    if isfile(telegrama_image_path):
        current_mesa_last_value = int(mesa_id[-2])

        if current_mesa_last_value == 0:
            item_dataset_type = 2 #test
        elif current_mesa_last_value == 9:
            item_dataset_type = 1 #validation
        else:
            item_dataset_type = 0 #train
        item_dataset_type = types_dataset[item_dataset_type]

        image_save_filename = mesa_id + '.tiff'

        item = {
            "file_name": mesa_id + '.jpg',
            "ground_truth": {
                "gt_parse": dict()
            }
        }

        for candidato_nombre in ("massa", "bullrich", "milei", "bregman", "schiaretti"):
            item["ground_truth"]["gt_parse"][candidato_nombre] = str(row.get(candidato_nombre))

        dataset_items[item_dataset_type].append(item)
        save_image_path = f"{types_dataset_paths[item_dataset_type]}{image_save_filename}"

        telegrama_tiff = PilImage.open(telegrama_image_path)

        telegrama_tiff_width, telegrama_tiff_height = telegrama_tiff.size

        cropping_values_distrito = cropping_values_mapping[distrito]

        crop_left = int(telegrama_tiff_width * cropping_values_distrito[0])
        crop_right = int(telegrama_tiff_width * cropping_values_distrito[1])
        crop_top = int(telegrama_tiff_height * cropping_values_distrito[2])
        crop_bottom = int(telegrama_tiff_height * cropping_values_distrito[3])

        cropped_telegrama = telegrama_tiff.crop((crop_left, crop_top, crop_right, crop_bottom))

        cropped_telegrama_original_size = cropped_telegrama.size
        ratio = float(output_image_size)/max(cropped_telegrama_original_size)
        new_size = tuple([int(x*ratio) for x in cropped_telegrama_original_size])
        cropped_telegrama = cropped_telegrama.resize(new_size)

        canvas = PilImage.new('1', (output_image_size, output_image_size) , (0))
        canvas.paste(cropped_telegrama, ((output_image_size-new_size[0])//2, (output_image_size-new_size[1])//2))

        canvas.save(save_image_path, optimize=True)

    else:
        telegrama_or_resultado_not_found.append(mesa_id)

for dataset_type in dataset_items.keys():
    print('length', dataset_type, len(dataset_items[dataset_type]), len(dataset_items[dataset_type])/limit)

for dataset_type in dataset_items.keys():
    with open(types_dataset_paths[dataset_type] + 'metadata.jsonl', 'w') as fp:    
        for item in dataset_items[dataset_type]:
            line = {"file_name": item["file_name"], "ground_truth": json.dumps(item["ground_truth"])}
            fp.write(json.dumps(line) + "\n")


In [ ]:
compress_directory_to_zip('data/donut_dataset/', 'data/donut_dataset_elecciones.zip')